In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
MODEL_NAME = 'Resnet18'
LEARNING_RATE = 0.001
MODEL_PATH = 'weights/weights.pt'
LABEL_CSV_PATH = '../input/petfinder-pawpularity-score/train.csv'
TRAIN_PATH = '../input/petfinder-pawpularity-score/train'
TEST_PATH = '../input/petfinder-pawpularity-score/test'
BATCH_SIZE = 512
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
from torch.utils import data
from PIL import Image
from torchvision import transforms
import pandas as pd


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    # transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  # 标准化至[-1,1]
])


# 定义自己的数据集合
class FlameSet(data.Dataset):
    def __init__(self, root):
        # 所有图片的绝对路径
        self.imgs_name = os.listdir(root)
        self.imgs = [os.path.join(root, k) for k in self.imgs_name]
        self.transforms = transform
        data = pd.read_csv(LABEL_CSV_PATH)
        self.Y = {k: v for k, v in zip(data.iloc[:, 0], data.iloc[:, -1])}

    def __getitem__(self, index):
        img_path = self.imgs[index]
        imgs_name = self.imgs_name[index]
        label = self.Y.get(imgs_name[:-4])
        pil_img = Image.open(img_path)
        if self.transforms:
            data = self.transforms(pil_img)
        else:
            pil_img = np.asarray(pil_img)
            data = torch.from_numpy(pil_img)

        return data, label

    def __len__(self):
        return len(self.imgs)

data_set = FlameSet(TRAIN_PATH)

train_loader = torch.utils.data.DataLoader(data_set, batch_size = BATCH_SIZE, shuffle=True)

# -------Test data------
class TestSet(data.Dataset):
    def __init__(self, root):
        # 所有图片的绝对路径
        self.imgs_name = os.listdir(root)
        self.imgs = [os.path.join(root, k) for k in self.imgs_name]
        self.transforms = transform

    def __getitem__(self, index):
        img_path = self.imgs[index]
        imgs_name = self.imgs_name[index]
        pil_img = Image.open(img_path)
        if self.transforms:
            data = self.transforms(pil_img)
        else:
            pil_img = np.asarray(pil_img)
            data = torch.from_numpy(pil_img)

        return data, imgs_name[:-4]

    def __len__(self):
        return len(self.imgs)

test_set = TestSet(TEST_PATH)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = 64, shuffle=True)

In [ ]:
import sys
import time

# _, term_width = os.popen('stty size', 'r').read().split()
term_width = int(60)

TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time


def progress_bar(current, total, msg=None):
    global last_time, begin_time
    if current == 0:
        begin_time = time.time()  # Reset for new bar.

    cur_len = int(TOTAL_BAR_LENGTH * current / total)
    rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

    sys.stdout.write(' [')
    for i in range(cur_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    cur_time = time.time()
    step_time = cur_time - last_time
    last_time = cur_time
    tot_time = cur_time - begin_time

    L = []
    L.append('  Step: %s' % format_time(step_time))
    L.append(' | Tot: %s' % format_time(tot_time))
    if msg:
        L.append(' | ' + msg)

    msg = ''.join(L)
    sys.stdout.write(msg)
    for i in range(term_width - int(TOTAL_BAR_LENGTH) - len(msg) - 3):
        sys.stdout.write(' ')

    # Go back to the center of the bar.
    for i in range(term_width - int(TOTAL_BAR_LENGTH / 2) + 2):
        sys.stdout.write('\b')
    sys.stdout.write(' %d/%d ' % (current + 1, total))

    if current < total - 1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()


def format_time(seconds):
    days = int(seconds / 3600 / 24)
    seconds = seconds - days * 3600 * 24
    hours = int(seconds / 3600)
    seconds = seconds - hours * 3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes * 60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds * 1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis).zfill(3) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

In [ ]:
from torch import nn, optim

class Trainer(object):

    def __init__(self, model_name, model, lr):
        """
        初始化
        :param model_name: 模型名字
        :param model: model对象
        :param lr: 学习率
        """
        self.model_name = model_name
        self.model = model
        self.lr = lr
        self.best_epoch = 0
        self.best_loss = 500

        self.model = self.model.to(DEVICE)

        self.optimizer = optim.SGD(
            self.model.parameters(),
            self.lr,
            momentum=0.9,
            weight_decay=5e-4)


    def train(self, trainloader):
        self.model.train().to(DEVICE, non_blocking=True).float()

        train_loss, total = 0, 0
        criterion = nn.MSELoss().to(DEVICE)

        for idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(DEVICE, non_blocking=True).float(), targets.to(DEVICE, non_blocking=True).float()
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            outputs = outputs.view(-1).to(torch.float32).to(DEVICE)
            targets = targets.to(torch.float32).to(DEVICE)
            targets = targets * 0.01
            loss = criterion(outputs, targets).to(DEVICE)
#             print('predict ->', outputs)
#             print('targets ->', targets)
            print('loss ->', loss)
#             print('MSE loss ->', ((outputs - targets)**2).mean() * 10000)
            loss.backward()

            self.optimizer.step()

            train_loss += loss.item()
            total += targets.size(0)

            progress_bar(
                idx, len(trainloader), 'Loss: %.3f ' % (train_loss / (idx + 1)))


    def evaluate(self, epoch, testloader):
        self.save_model(self.model, self.model_name, 666, epoch)

    def save_model(self, model, model_name, best_loss, epoch):
        state = {
            'net': model.state_dict(),
            'best_loss': best_loss,
            'epoch': epoch,
        }

        save_name = os.path.join('weights', 'weights50.pt')

        if not os.path.exists(os.path.dirname(save_name)):
            os.makedirs(os.path.dirname(save_name))

        torch.save(state, save_name)
        print("\nSaved state at %.03f%% ." %
              (best_loss))
        self.best_epoch = epoch
        self.best_loss = best_loss

    def load_model(self, path=None):
        """
        Load previously saved model. THis doesn't check for precesion type.
        """
        if path is not None:
            checkpoint_name = path


        if not os.path.exists(checkpoint_name):
            print("Best model not found")
            return

        checkpoint = torch.load(checkpoint_name, map_location=torch.device(DEVICE))
        self.model.load_state_dict(checkpoint['net'])
        self.best_epoch = checkpoint['epoch']
        self.best_loss = checkpoint['best_loss']
        print("Loaded Model with loss: %.3f%%, from epoch: %d" %
              (checkpoint['best_loss'], checkpoint['epoch'] + 1))

    def train_and_evaluate(self, traindataloader, testdataloader, no_of_steps):
        for i in range(no_of_steps):
            self.best_loss = 500
            print('\nEpoch: %d' % (i + 1))
            self.train(traindataloader)
#             self.evaluate(i, testdataloader)

    def predict(self, test_loader):
        self.model.eval()
        for idx, (inputs, name) in enumerate(test_loader):
            inputs = inputs.to(DEVICE)
            outputs = self.model(inputs)
            outputs = outputs.view(-1).to(torch.float32)
            ans = pd.DataFrame()
            tmp = outputs.cpu()
            ans.insert(0, 'Id', name)
            ans.insert(1, 'Pawpularity', tmp.detach().numpy())
            print(ans.head())
            ans.to_csv('submission.csv', index=False)

In [ ]:
from torchvision import models

model_name = MODEL_NAME
steps = 50
model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False

num_fits = model_ft.fc.in_features  # 512
# model_ft.fc = nn.Linear(num_fits, 1)
model_ft.fc_ = model_ft.fc

model_ft.fc = nn.Sequential(
            nn.Linear(num_fits, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 1)
)

print('\n Model: {} | Eopch: {} | Learning rate: {} '.format(model_name, steps, LEARNING_RATE))
# model_ft = model_ft.to(DEVICE, non_blocking=True).float()
trainer = Trainer(model_name, model_ft, LEARNING_RATE)
trainer.train_and_evaluate(train_loader, train_loader, steps)
trainer.evaluate(1, test_loader)
# trainer.predict(test_loader)